In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [66]:
# 인자로 들어가는 url 예시 
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=31794
def getReview(url):
    code = url[-6:]
    if code[0] == '=':
        code = code[1:]
    base1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
    base2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
    reviewUrl = base1 + code + base2
    
    # 총 리뷰 수
    response = requests.get(reviewUrl.format(1))
    soup = BeautifulSoup(response.text, 'html.parser')
    numOfReviews = soup.find('div', {'class':'score_total'}).em.text
    numOfReviews = "".join(c for c in numOfReviews if c != ',')
    numOfReviews = int(numOfReviews)
    
    #  평점, 리뷰
    def getData(url):
        response = requests.get(url.format(1))
        soup = BeautifulSoup(response.text, 'html.parser')
        s, t = [], []
        for li in soup.find('div', {'class':'score_result'}).find_all('li'):
            s.append(int(li.em.text))
            temp = "".join(c for c in li.p.text if c not in ['\r', '\n', '\r'])
            t.append(temp)
        return s, t
    
    score, text = [], []
    # 반복 횟 수
    numIter = numOfReviews // 10 + numOfReviews % 10
    for i in range(1, numIter):
        s, t = getData(reviewUrl + str(i))
        score += s
        text += t
        if i % 5 == 0:
            print("{}페이지..".format(i), end='\r')
    # csv 파일로 저장
    df = pd.DataFrame([score, text]).T
    df.columns = ['score', 'text']
    df.to_csv(code + '.csv', encoding='ms949')    

In [63]:
getReview('https://movie.naver.com/movie/bi/mi/point.nhn?code=10011345')